In [ ]:
import glob 
import pandas as pd
from rapidfuzz import fuzz
import pycountry
import pandas as pd
import numpy as np


In [ ]:
RAW_MATCH_DETAILS_DIR = "../Data/Raw/Match_details"

DNF_KEYWORDS = ['WO', 'INJ', 'RET', 'DSQ', 'DNS']
DNF_PATTERN = '|'.join(DNF_KEYWORDS)
DNF_PATTERN_CAPTURE = r'(' + r'|'.join(DNF_KEYWORDS) + r')'

TEAM_NAME_PATTERN = r'team|federacion|federation|table'

DROP_COLUMNS_START = ["resultStatus", "tableName", "tableNumber","venueName"]

In [ ]:
# Get all match files in the RAW_MATCH_DETAILS_DIR
# Create an all_matches_df to be filtered down 
# Many of these matches are in fact match ups between teams rather than players - need to filter out.
# some of the singles matches are from teams matches and events - can keep these.



# parse all match details inside the json files.
all_match_details_files = glob.glob(os.path.join(RAW_MATCH_DETAILS_DIR, "*.json"))
all_matches = []
for file in all_match_details_files:
    with open(file, 'r', encoding='utf-8') as f:
        matches_data = json.load(f)

    all_matches.extend(matches_data)

# create the df 

all_matches_df = pd.DataFrame(all_matches)







In [ ]:
# Keep here inside this cell to prevent rerunning file parsing and extraction every time.
all_matches_df = pd.DataFrame(all_matches)

# drop some columns that are not currently of interest for the project (e.g table number and venue etc )
all_matches_df.drop(columns=DROP_COLUMNS_START, inplace=True)


# initialise the cleaned matches df and drop empty columns and na rows
print(f"all_matches_df before dropping empty rows and columns: {len(all_matches_df)}")
cleaned_matches_df = all_matches_df.dropna(axis=0, how='all', inplace=False)
cleaned_matches_df = all_matches_df.dropna(axis=0, how='all', inplace=False)
print(f"cleaned_matches_df after dropping empty rows and columns: {len(cleaned_matches_df)}")


dnf_from_overallScores = cleaned_matches_df["overallScores"].str.extract(DNF_PATTERN_CAPTURE, expand=False).str.strip()
dnf_from_resultoverallScores = cleaned_matches_df["resultOverallScores"].str.extract(DNF_PATTERN_CAPTURE, expand=False).str.strip()

cleaned_matches_df["dnf"] = dnf_from_overallScores
cleaned_matches_df["dnf"] = cleaned_matches_df["dnf"].fillna(dnf_from_resultoverallScores)
print("✅ 'dnf' column updated by sequentially filling missing values from 'overallScores' THEN 'resultOverallScores'.")




all_matches_df before dropping empty rows and columns: 24469
cleaned_matches_df after dropping empty rows and columns: 24469
✅ 'dnf' column updated by sequentially filling missing values from 'overallScores' THEN 'resultOverallScores'.


In [ ]:
# Before filtering - extact key information from the 'competitors' column
# player name column can contain team names.
# competitors column keeps track of either the 1 player for a singles listing
# or the multiple players for a team listing 
# some doubles matches may be leftover from payload filtering - need to filter out.
# some doubles matches may be here inside team events - need to filter out.

def extract_competitor_details(competitor_list):
    """
    Extracts only the top-level competitor details (Name, ID, ORG) 
    for Home (H) and Away (A) competitors, ignoring the nested 'players' dict.
    """
    
    # Initialize the output dictionary  
    data = {}
    
    # check for empty data incase 
    if not isinstance(competitor_list, list) or len(competitor_list) < 2:
        return pd.Series(data)

    try:
        # use prefix pattern to determine home/away and build the keys for the output dict.
        for comp in competitor_list:
            comp_type = comp.get('competitorType')
            
            if comp_type == 'H':
                prefix = 'home'
            elif comp_type == 'A':
                prefix = 'away'
            else:
                continue                
           
            
            # get competitor id 
            data[f'{prefix}CompetitorId'] = comp.get('competitiorId', pd.NA)
            
            # get competitor name 
            data[f'{prefix}CompetitorName'] = comp.get('competitiorName', pd.NA)
            
            # get competitor country code 
            data[f'{prefix}CompetitorOrg'] = comp.get('competitiorOrg', pd.NA)

            data[f'{prefix}Player(s)'] = [player.get('playerName', pd.NA) for player in comp.get('players', pd.NA)]

            data[f'{prefix}PlayerGameScores'] = comp.get('scores', pd.NA)



    except Exception as e:
        print(f"Error processing row: {e} | Data: {competitor_list}")
        pass

    return pd.Series(data)

#
print("--- 🚀 Getting competitor details 🚀 ---")

# apply the function to the competitors column from the main df
competitor_details_df = cleaned_matches_df['competitiors'].apply(extract_competitor_details)



cleaned_matches_df = pd.concat([cleaned_matches_df, competitor_details_df], axis=1)
cleaned_matches_df.drop(columns=["competitiors"],inplace=True, errors='ignore')

print("✅ Competitor details extracted and added to cleaned_matches_df and competitiors column dropped.")


--- 🚀 Getting competitor details 🚀 ---


In [ ]:
print(f"cleaned_df before dropping para and age limit matches: {len(cleaned_matches_df)}")
age_limit_mask = cleaned_matches_df['subEventName'].str.contains(r"U\d{2}", case=False, na=False)
para_class_mask = cleaned_matches_df['subEventName'].str.contains("class", case=False, na=False)
age_para_filter = age_limit_mask | para_class_mask
cleaned_matches_df = cleaned_matches_df[~age_para_filter].copy()


print(f"cleaned_df after dropping para and age limit matches: {len(cleaned_matches_df)}")



cleaned_df before dropping para and age limit matches: 24469
cleaned_df after dropping para and age limit matches: 24295


In [ ]:
print(f"cleaned_df before dropping names with teams_parent_data: {len(cleaned_matches_df)}")
team_parent_filter = cleaned_matches_df["teamParentData"].notna()
team_parent_df  = cleaned_matches_df[team_parent_filter].copy()
cleaned_matches_df = cleaned_matches_df[~team_parent_filter ].copy()
print(f"cleaned_df after dropping names with teams_parent_data: {len(cleaned_matches_df)}")

cleaned_df before dropping names with teams_parent_data: 24295
cleaned_df after dropping names with teams_parent_data: 23494


In [ ]:
print(f"cleaned_df before dropping names with numbers (teams): {len(cleaned_matches_df)}")
home_contains_digit = cleaned_matches_df["homeCompetitorName"].str.contains(r"[0-9]", na=False)
away_contains_digit = cleaned_matches_df["awayCompetitorName"].str.contains(r"[0-9]", na=False)
keep_filter = ~(home_contains_digit | away_contains_digit)
cleaned_matches_df = cleaned_matches_df[keep_filter].copy()
print(f"cleaned_df after dropping names with numbers (teams): {len(cleaned_matches_df)}")

cleaned_df before dropping names with numbers (teams): 23494
cleaned_df after dropping names with numbers (teams): 23494


In [ ]:
print(f"cleaned_df before dropping matches with multiple players (teams): {len(cleaned_matches_df)}")
home_multiple_players = cleaned_matches_df["homePlayer(s)"].map(len) >1
away_multiple_players = cleaned_matches_df["awayPlayer(s)"].map(len) >1
keep_filter = ~(home_multiple_players | away_multiple_players)
remove_filter = (home_multiple_players | away_multiple_players)
remove_df = cleaned_matches_df[remove_filter].copy()
cleaned_matches_df = cleaned_matches_df[keep_filter].copy()
print(f"cleaned_df after dropping matches with multiple players (teams): {len(cleaned_matches_df)}")

cleaned_df before dropping matches with multiple players (teams): 23494
cleaned_df after dropping matches with multiple players (teams): 23494


In [ ]:
def build_country_name_list():
    """
    Creates a comprehensive list of country names for fuzzy matching from pycountries
    """
    country_names = set()
    
    #  Get ALL official names and common names from pycountry
    for country in pycountry.countries:
        # Standard Common Name
        country_names.add(country.name)
        # Official Full Name (often different)
        if hasattr(country, 'official_name'):
            country_names.add(country.official_name)
        # Historical/Alternative Names (if the library provides them)
        if hasattr(country, 'common_name'):
             country_names.add(country.common_name)

    # Add other possible options for sports teams across different tournaments and regions
    # generated using Google Gemini - may be superfluous or be missing some possible options
    
    sports_variants = [
        # China/Taiwan/HK
        'Chinese Taipei', 'Taiwan', 'Hong Kong, China', 'Hong Kong','Macau, China', 'Macao'
        # N/S Korea
        'Republic of Korea', 'North Korea', 'South Korea', 'DPR Korea',
        # Former Czech/Slovak
        'Czechia', 'Czech Republic', 'Slovakia',
        # Common English/French alternatives
        'Ivory Coast', 'Cote d\'Ivoire', 'Cape Verde', 'Cabo Verde',
        # Common abbreviations that might appear unparsed
        'DR Congo', 'ROC', 'PRC', 'USA', 'UK', 'UAE' 
    ]
    
    country_names.update(sports_variants)
    
    # Clean up the list (remove duplicates and empty/None entries)
    final_list = [name.strip() for name in country_names if name and isinstance(name, str)]
    
    return final_list

# Generate the master list
list_of_countries = build_country_name_list()

print(f"Generated a master list of {len(list_of_countries)} country/territory names and variants for fuzzy matching.")

def fuzz_match_check(player_name, keywords_list, threshold):
    """
    Checks if a player name has a high fuzzy match score against any
    keyword in the list.
    """
    # Safety check for empty/NA names
    if pd.isna(player_name):
        return False
        
    player_name_lower = str(player_name).lower()
    
    for keyword in keywords_list:
        # We use partial_ratio to see if the keyword is 'contained'
        # in the player name with high confidence.
        score = fuzz.ratio(player_name_lower, keyword.lower())
        
        # If any keyword scores above the threshold, flag it as suspicious
        if score >= threshold:
            return True
            
    # If no keyword matched, it's a safe name
    return False

print(f"cleaned_df before dropping matches with countries as 'player names' (teams): {len(cleaned_matches_df)}")

threshold = 90

home_is_fuzzy = cleaned_matches_df['homeCompetitorName'].apply(fuzz_match_check,args=(list_of_countries,threshold))
away_is_fuzzy = cleaned_matches_df['awayCompetitorName'].apply(fuzz_match_check,args=(list_of_countries,threshold))

remove_check_filter = home_is_fuzzy | away_is_fuzzy
remove_check_df = cleaned_matches_df[remove_check_filter].copy()
cleaned_matches_df = cleaned_matches_df[~remove_check_filter]
print(f"cleaned_df after dropping matches with countries as 'player names' (teams): {len(cleaned_matches_df)}")




Generated a master list of 439 country/territory names and variants for fuzzy matching.
cleaned_df before dropping matches with countries as 'player names' (teams): 23494
cleaned_df after dropping matches with countries as 'player names' (teams): 23494


In [ ]:
# All teams matches should be removed by now 

def extract_format(config):
    """
    Attempts to extract the best of format from the 'matchConfig' column
    """
    data = {}
    if not isinstance(config,dict):
        return pd.Series(data)
    try:
        data['bestOf':] = config.get('bestOfXGames')
        data['ttrReview'] = config.get('tTRReview')
        return pd.Series(data)
    except Exception as e:
        print(f"Error processing row: {e} | Data: {config}")
        pass
    return pd.Series(data)
#
print("--- 🚀 Getting Match Config deatils 🚀 ---")

# apply the function to the competitors column from the main df
match_config_df = cleaned_matches_df['matchConfig'].apply(extract_format)


cleaned_matches_df = pd.concat([cleaned_matches_df, match_config_df], axis=1)
cleaned_matches_df.drop(columns=["matchConfig"],inplace=True, errors='ignore')

print("✅ Match config extracted and added to cleaned_matches_df,matchConfig column dropped.")


--- 🚀 Getting Match Config deatils 🚀 ---
✅ Match config extracted and added to cleaned_matches_df,matchConfig column dropped.


In [ ]:
# """
# After this filtering out - all matches left should be single player matches.
# The following checks can bed done on top of this:
# """
# # #### Check for players / teams with id length != 6 (6  appears to be the standard legnth but not for all players )###
# # home_id_length_check = cleaned_matches_df["homeCompetitorId"].astype(str).map(len) != 6
# # away_id_length_check = cleaned_matches_df["awayCompetitorId"].astype(str).map(len) != 6 
# # id_check_filter = home_id_length_check | away_id_length_check
# # id_check_df = cleaned_matches_df[id_check_filter].copy()
# # away_names = cleaned_matches_df["awayCompetitorName"].unique()
# # home_names = cleaned_matches_df["homeCompetitorName"].unique()
# # all_names = away_names.tolist() + home_names.tolist()
# # all_names = list(set(all_names))
# # al;_names.to_csv("all_names.csv", index=False)

# ### Check for matches where homeCompetitorName !- homePlayer(s) or the same case for away (any artefacts from competitors filtering) ###
# # home_check = cleaned_matches_df["homeCompetitorName"].str.lower() != cleaned_matches_df["homePlayer(s)"].str[0].str.lower()
# # away_check = cleaned_matches_df["awayCompetitorName"].str.lower() != cleaned_matches_df["awayPlayer(s)"].str[0].str.lower()
# # check_filter = home_check | away_check
# # names_match_check_df = cleaned_matches_df[check_filter].copy()
# # names_match_check_df.to_csv("names_match_check_df.csv", index=False)



# ### This filter removes mismatches where one value is simply blank (due to different reporting)###
# # blank_game_scores = cleaned_matches_df["gameScores"].isna()
# # blank_resultsGameScores = cleaned_matches_df["resultsGameScores"].isna()
# # both_blank_scores_df = cleaned_matches_df[blank_game_scores & blank_resultsGameScores].copy()
# # One game has NO scores at all (2345	TTEMSINGLES-----------R128001100----------	) can look up manually 
# # This game appears to be a walkover in favour of David Powell (away) due a +ve covid test for Pavel Sirucek
# # https://www.olympics.com.au/news/table-tennis-david-powell-fast-tracked-to-next-round/
# #blank_scores_df = cleaned_matches_df[blank_game_scores & blank_resultsGameScores].copy()



# ### Check for gameScores != resultsGameScores  (this should be 0 but not for team games ) ###
# ### normalise gameScores and gameScores to remove blank 0-0 values ###

# blank_game_scores = cleaned_matches_df["gameScores"].isna()
# blank_resultsGameScores = cleaned_matches_df["resultsGameScores"].isna()
# one_blank_score_filter = blank_game_scores | blank_resultsGameScores

# game_scores_check = cleaned_matches_df["gameScores"] != cleaned_matches_df["resultsGameScores"]
# normalised_gameScores = (
#     cleaned_matches_df['gameScores']
#     .astype(str)
#     .str.replace(r'(,0-0)+$', '', regex=True) # Remove trailing 0-0 sequences
#     .str.strip(',')                          # Remove any resulting trailing comma
# )
# normalised_resultGameScores = (
#     cleaned_matches_df['resultsGameScores']
#     .astype(str)
#     .str.replace(r'(,0-0)+$', '', regex=True) # Remove trailing 0-0 sequences
#     .str.strip(',')
# )

# score_mismatch_filter = (normalised_gameScores != normalised_resultGameScores)
# non_dnf_filter= cleaned_matches_df["dnf"].isna()
# non_blank_mismatch_filter = score_mismatch_filter & ~one_blank_score_filter & non_dnf_filter




# games_mismatch_df = cleaned_matches_df[non_blank_mismatch_filter].copy()
# games_mismatch_df[["eventId","documentCode","resultsGameScores", "gameScores","overallScores","resultOverallScores","dnf"]].to_csv("games_mismatch_df.csv", index=False)


    



In [ ]:
# Now to assess the issues if the same player (sameID) being given multiple names 
print("--- 🟢 Generating Name-to-ID Mapping for Review 🟢---")

# 1. Combine the ID and Name columns into one DataFrame (Home and Away)
home_map = cleaned_matches_df[['homeCompetitorId', 'homeCompetitorName']].rename(
    columns={'homeCompetitorId': 'competitor_id', 'homeCompetitorName': 'competitor_name'}
)
away_map = cleaned_matches_df[['awayCompetitorId', 'awayCompetitorName']].rename(
    columns={'awayCompetitorId': 'competitor_id', 'awayCompetitorName': 'competitor_name'}
)

# 2. Concatenate and drop duplicates to get a list of all unique ID-Name pairs
all_id_name_pairs = pd.concat([home_map, away_map]).dropna().drop_duplicates()

# 3. Group by ID and aggregate all associated names into a list
id_to_names_map = all_id_name_pairs.groupby('competitor_id')['competitor_name'].unique()

print("✅ Mapped all names to their unique competitor IDs.")
print(f"Total unique competitor IDs found: {len(id_to_names_map)}")

# 4. Filter for IDs that have MORE THAN ONE associated name (the problem cases)
# This finds where the list of unique names for one ID is greater than length 1
discrepancy_map = id_to_names_map[id_to_names_map.apply(len) > 1]

print(f"\nDiscrepancy Report: Found {len(discrepancy_map)} IDs with multiple names.")
print("You must inspect and choose a canonical name for these IDs:")
print(discrepancy_map.head(10))

--- 🟢 Generating Name-to-ID Mapping for Review 🟢---
✅ Mapped all names to their unique competitor IDs.
Total unique competitor IDs found: 2808

Discrepancy Report: Found 357 IDs with multiple names.
You must inspect and choose a canonical name for these IDs:
competitor_id
100001                   [ANTHONY Amalraj, Amalraj ANTHONY]
100032                 [ABDEL-AZIZ Farah, Farah ABDEL-AZIZ]
100189                     [ALAWLAQI Ahmed, Ahmed ALAWLAQI]
100439                           [SALEH Ahmed, Ahmed SALEH]
100486                           [ALTO Gaston, Gaston ALTO]
100621                     [Tiago APOLONIA, APOLONIA Tiago]
100696                             [Omar ASSAR, ASSAR Omar]
100868    [BALAZOVA Barbora, VARADY Barbora, Barbora BAL...
101192                     [BOBOCICA Mihai, Mihai BOBOCICA]
101480                       [CANTERO Jesus, Jesus CANTERO]
Name: competitor_name, dtype: object


In [ ]:
len(cleaned_matches_df)

23494